In [2]:
import cv2
import numpy as np
import os
os.chdir('./data')

In [3]:
cap = cv2.VideoCapture('1.mp4')
print(cap.get(3))
print(cap.get(4))

fourcc = cv2.VideoWriter_fourcc('X', 'V', 'I', 'D')
# out = cv2.VideoWriter('output.avi', fourcc, 30.0, (1280,720))

_, frame1 = cap.read()
_, frame2 = cap.read()

while cap.isOpened():
    # get all changes that happened between two frames
    diff = cv2.absdiff(frame1,frame2) # image segmentation
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)

    blur = cv2.GaussianBlur(gray, (5,5), 0)

    _, thresh = cv2.threshold(blur, 50, 255, cv2.THRESH_BINARY)

    dilation = cv2.dilate(thresh, None, iterations=10)

    contours,_ = cv2.findContours(dilation, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # draw_contours = cv2.drawContours(frame1, contours, -1, (0,255,0), 2) # -1 means draw all contours   2 means thickness of the contour

    for contour in contours:
        (x,y,w,h) = cv2.boundingRect(contour)

        if cv2.contourArea(contour) < 1000:
            continue

        cv2.rectangle(frame1, (x,y), (x+w, y+h), (0,255,0), 2)
        cv2.putText(frame1, "Status: Movement", (10,20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 3)

    img = cv2.resize(frame1, (1280,720))
    # out.write(img)
    cv2.imshow("feed", frame1)
    # cv2.imshow('diff',diff)
    # cv2.imshow('gray',gray)
    # cv2.imshow('blur', blur)
    # cv2.imshow('thresh', thresh)
    # cv2.imshow('dilation', dilation)

    frame1 = frame2
    _, frame2 = cap.read()

    if cv2.waitKey(60) == 27: # wait 60 seconds before move to next frame
        break

cv2.destroyAllWindows()
cap.release()
# out.release()

1280.0
720.0


QSocketNotifier: Can only be used with threads started with QThread


<h1>with counter</h1>

In [4]:
def get_center(x,y,w,h):
    x1 = w//2
    y1 = h//2
    cx = x + x1
    cy = y + y1
    return cx,cy


centers = []
cars = 0


cap = cv2.VideoCapture('1.mp4')

# same as diff between frame1 and frame2
BGS = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=100)

while True:
    _,frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (3,3), 0)
    mask = BGS.apply(blur)
    # img, kernel, iterations
    dilation = cv2.dilate(mask, np.ones((3,3), np.uint8), iterations=3)

    contours, _ = cv2.findContours(dilation, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    cv2.line(frame,(10, 450), (1280, 450), (255, 0, 0), 2)

    for i,contour in enumerate(contours):
        (x, y, w, h) = cv2.boundingRect(contour)
        valid_contour = (w >= 80) and (h >= 80)
        if not valid_contour:
            continue

        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        center = get_center(x, y, w, h)
        centers.append(center)
        cv2.circle(frame, center, 4, (0, 0, 255), -1)

        for x,y in centers:
            if y < (450+10) and y > (450-10):
                cars += 1
                cv2.line(frame,(10, 450), (1280, 450), (0, 255, 0), 2)
                centers.remove((x,y))
                print(f"num of cars -> {cars}")

    cv2.putText(frame, "cars -> " + str(cars), (500, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 5)
    cv2.imshow("video", frame)
    # cv2.imshow("mask", mask)
    # cv2.imshow("dilation", dilation)
    key = cv2.waitKey(30)
    if key == 27:
        break

cv2.destroyAllWindows()

num of cars -> 1
num of cars -> 2
num of cars -> 3
num of cars -> 4
num of cars -> 5
